**About** : This notebook is used to infer models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

### Imports

In [ ]:
import os
import gc
import sys
import cv2
import glob
import json
import torch
import shutil
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tqdm import tqdm
from joblib import Parallel, delayed

In [ ]:
from util.logger import Config
from util.metrics import macro_auc
from util.torch import load_model_weights

from data.dataset import WaveInfDataset
from data.preparation import prepare_data, prepare_folds
from data.processing import create_target_path, ProgressParallel, get_load_librosa_save_h5py

from model_zoo.models import define_model
from inference.predict import predict

from params import CLASSES

### Params

In [ ]:
EVAL = False

In [ ]:
if EVAL:
    DATA_PATH = "../input/train_audio/"
else:
    DATA_PATH = "../input/unlabeled_soundscapes/"

In [ ]:
BATCH_SIZE = 32
USE_FP16 = True
NUM_WORKERS = 2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DURATION = 5
SR = 32000

In [ ]:
FOLD = 0 if EVAL else "fullfit_0"

EXP_FOLDERS = [
    # ("../logs/2024-04-12/8/", [FOLD]),   # LB 0.64 baseline
    ("../logs/2024-04-18/12/", [FOLD]),  #
]

In [ ]:
# from util.logger import upload_to_kaggle

# upload_to_kaggle(
#     [f for f, _ in EXP_FOLDERS],
#     directory="../output/dataset_1/",
#     dataset_name="BirdCLEF 2024 Weights 1",
#     update_folders=True
# )

### Preprocessing

In [ ]:
if EVAL:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*/*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])

    folds = pd.read_csv('../input/folds_4.csv')
    folds['id'] = folds['filename'].apply(lambda x: x.split('/')[-1][:-4])
    df = df.merge(folds)
    df = df[df['fold'] == 0].reset_index(drop=True)

    df["primary_label"] = df["path"].apply(lambda x:  x.split('/')[-2])
else:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])

df["duration"] = df["path"].apply(lambda x: librosa.get_duration(path=x))

df['slice'] = df['duration'].apply(lambda x: [(DURATION * i * SR, DURATION * (i + 1) * SR) for i in range(int(np.ceil(x / DURATION)))])
df = df.explode("slice")

df["t_end"] = DURATION
df['t_end'] = df[["id", "t_end"]].groupby("id").cumsum()

if EVAL:
    df = df[df['t_end'] == 5]

### Models

In [ ]:
models = []

for exp_folder, folds in EXP_FOLDERS:
    models_ = []
    config = Config(json.load(open(exp_folder + "config.json", "r")))

    model = define_model(
        config.name,
        config.melspec_config,
        head=config.head,
        aug_config=config.aug_config,
        num_classes=config.num_classes,
        n_channels=config.n_channels,
        drop_rate=config.drop_rate,
        drop_path_rate=config.drop_path_rate,
        increase_stride=config.increase_stride,
        verbose=True,
        pretrained=False
    )
    model = model.to(DEVICE).eval()
    
    for fold in folds:
        weights = exp_folder + f"{config.name}_{fold}.pt"
        model = load_model_weights(model, weights, verbose=config.local_rank == 0)
        models_.append(model)
        
    models.append(models_)

In [ ]:
dataset = WaveInfDataset(
    df,
    normalize=config.normalize,
    max_len=config.melspec_config["sample_rate"] * config.duration,
)

In [ ]:
preds = predict(
    model,
    dataset,
    config.loss_config,
    batch_size=BATCH_SIZE,
    device=DEVICE,
    use_fp16=USE_FP16,
    num_workers=NUM_WORKERS,
)

In [ ]:
if EVAL:
    auc = macro_auc(df["primary_label"].values.tolist(), preds)
    print(f'Fold 0 AUC: {auc:.3f}')

In [ ]:
if EVAL:
    from data.preparation import prepare_data
    from data.dataset import WaveDataset

    df_ = prepare_data()
    df_ = df_[df_['fold'] == 0].reset_index(drop=True)
    df = df.merge(df_[["filename", "secondary_labels"]], how="left")
    df['path_ft'] = df['path']
    dataset = WaveDataset(df)

    y, y_s = dataset.get_targets()

    auc = macro_auc(y, preds)
    print(f'Fold 0 AUC:\t {auc:.3f}')
    
    auc = macro_auc(y_s, preds)
    print(f'Fold 0 AUC_s:\t {auc:.3f}')

In [ ]:
if not EVAL:
    sub = pd.DataFrame((df["id"] + "_" + df["t_end"].astype(str)).values, columns=['row_id'])
    sub.loc[:, CLASSES] = preds
    sub.to_csv('../output/tmp/submission.csv', index=False)
    sub.head()

### EDA

In [ ]:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)

from util.plots import display_audio

#### Max per class

In [ ]:
max_df = pd.DataFrame(sub.max()).T
max_df = max_df[max_df.columns[1:]].astype(float)
max_df

In [ ]:
sns.distplot(max_df.values[0])
plt.show()

#### Max per recording

In [ ]:
dfg = sub.copy()
dfg['row_id'] = dfg['row_id'].apply(lambda x:x.split('_')[0])
dfg = dfg.groupby('row_id').max()
dfg = dfg.max(1)


pd.DataFrame(dfg[dfg < 0.3]).T

In [ ]:
dfg = sub.copy()

dfg['row_id'] = dfg['row_id'].apply(lambda x:x.split('_')[0])

dfg['max'] = dfg[dfg.columns[1:]].max(1)
dfg_ = dfg[["row_id", "max"]].groupby('row_id').max().reset_index()

In [ ]:
dfg = dfg.drop('max', axis=1).merge(dfg_, how="left")

dfg[CLASSES] = preds / dfg['max'].values[:, None]

# pd.DataFrame(dfg).sort_values("max")

In [ ]:
sns.histplot(dfg.values)
plt.show()

In [ ]:
val = np.load(EXP_FOLDERS[0][0] + "pred_val_0.npy")
sns.histplot(val.max(-1))
plt.show()

#### Single sample

In [ ]:
id_ = "1051136855"
id_ = "1227419659"

df_id = sub[sub["row_id"].apply(lambda x: id_ in x)]

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df_id[df_id.columns[1:]].max(1).values)
plt.grid()
plt.xticks([i for i in range(0, len(df_id), 2)], [f"{i * 5 // 60}'{i * 5 % 60}" for i in range(0, len(df_id), 2)])
plt.show()

In [ ]:
display_audio(DATA_PATH + f"{id_}.ogg", duration=None)


Done ! 